# En este colab se prueba el modelo finetuneado de BNE con text encodings sobre un conjunto de conversaciones de prueba. Por último, se puede utilizar una función que genera aleatoriamente una conversación desde unos dataset de prueba para validación

*0*. Se instalan las bibliotecas necesarias

In [ ]:
!pip install datasets
!pip install transformers

*1*. Se importan las dependencias

In [ ]:
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import datasets
import numpy as np
import random

from transformers import  RobertaForSequenceClassification
from transformers import RobertaTokenizer

from sklearn.metrics import accuracy_score

In [ ]:
torch.cuda.empty_cache()
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

*2*. Definimos el modelo a utilizar, en este caso el modelo finetuneado de BNE con text enconding

In [ ]:
MODEL_TYPE = "PlanTL-GOB-ES/roberta-large-bne-te"

In [ ]:
MAX_LEN=256

*3*. TestDataset and CompDataset son las clases usadas para la tokenización de la frases, CompDataset se utiliza para entrenar (devuelve también el label), y TestDataset es utilizada para tokenizar las frases para validación

In [ ]:
class TestDataset(Dataset):

    def __init__(self, df):
        self.df_data = df

    def __getitem__(self, index):
        sentence1 = self.df_data.loc[index, 'premise']
        sentence2 = self.df_data.loc[index, 'hypothesis']

        encoded_dict = tokenizer.encode_plus(
                    sentence1, sentence2,
                    add_special_tokens = True, 
                    max_length = MAX_LEN, 
                    pad_to_max_length = True,
                    return_attention_mask = True,  
                    return_tensors = 'pt',    
                    padding="max_length", 
                    truncation=True
               )
        
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]

        sample = (padded_token_list, att_mask)
        return sample

    def __len__(self):
        return len(self.df_data)

In [ ]:
class CompDataset(Dataset):

    def __init__(self, df):
        self.df_data = df

    def __getitem__(self, index):
        sentence1 = self.df_data.loc[index, 'premise']
        sentence2 = self.df_data.loc[index, 'hypothesis']

        encoded_dict = tokenizer.encode_plus(
                    sentence1, sentence2,   
                    add_special_tokens = True,  
                    max_length = MAX_LEN,        
                    pad_to_max_length = True,
                    return_attention_mask = True,
                    return_tensors = 'pt', 
                    padding="max_length", 
                    truncation=True
               )
        
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        
        target = torch.tensor(self.df_data.loc[index, 'label'])

        sample = (padded_token_list, att_mask, target)
        return sample

    def __len__(self):
        return len(self.df_data) 

*4*. Se carga el modelo

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained(MODEL_TYPE)
model =  RobertaForSequenceClassification.from_pretrained(MODEL_TYPE, num_labels=3)

model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50262, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
         

*5*. Se define un conjunto de frases para validar el algoritmo sobre conversaciones de prueba

In [ ]:
frases_saludos = ["Buen día", 
"Cómo se encuentra el día de hoy?",
"espero que no llame más hasta el día del juicio final",
"es un día lluvioso",
"Le doy la bienvenida a nuestro centro de ayuda",
"espero se encuentre bien el día de hoy",
"son buenos los días en verano",
"es un gusto saludarlo",
"hace calor hoy",
"buenas tardes",
"buenas noches",
"buenos días",
"como está?",
"muy buenos días",
"muy buenas tardes",
"muy buenas noches",
"excelente día",
"que tal?",
"que tal está?",
"cómo estás?",
 "cómo le va?",
"cómo te va?",
"cómo está usted",
"que gusto poder atenderlo",
"espero se encuentre bien",
"hola",
"hoy no es mi día",
"he tenido un día fatal",
"cómo está el helado?",
"como pudo llegar el día?"]

frases_ayuda = ["Cómo lo puedo ayudar ",
"Cómo se encuentra el día de hoy?",
"qué difícil es la tarea de atender al público",
"cómo lo puedo matar?",
"le doy la bienvenida",
"espero poder ayudarlo el día de hoy",
"en qué podemos ayudarlo en este momento?",
"es un gusto atender su consulta",
"no es un gusto atender su consulta",
"hay muchas consultas hoy",
"es un gusto asistirle, dígame cómo",
"en qué puedo servirle?",
"qué necesita? ",
"estoy a su disposición para asistirle",
"por favor dígame como lo puedo ayudar?",
"qué tengo que hacer para que no llame más?"]


In [ ]:
frases_cliente=["Hola, mi nombre es Pedro","Necesito asistencia con un seguro"]

> make_conversation es una función auxilar para crear una conversación a partir de las frases anteriores



In [ ]:
def make_conversation():
  frases_operadora=[]
  frases_operadora.append(random.choice(frases_saludos))
  frases_operadora.append(random.choice(frases_ayuda))

  for i in range(max(len(frases_operadora),len(frases_cliente))):
    if i<len(frases_operadora):
      print("Operadora: "+ frases_operadora[i])
    if i<len(frases_cliente):
      print("  Cliente: "+ frases_cliente[i])

  return frases_operadora,frases_cliente   

*6* La función getlabel predice el entailment de una frase, que la recibe como una TestDataset 

In [ ]:
def get_label(test_dataloader):
  total = 0

  for _, batch in enumerate(test_dataloader):

    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)  

    import os
    os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
    outputs = model(b_input_ids, attention_mask=b_input_mask)
    #print(outputs)
    preds = outputs[0].detach().cpu().numpy()

    y_pred = np.argmax(preds, axis=1)
  
  return y_pred

 > predict_matrix calcula una matriz de entailments, indicando en la posición (i,j) el entailment de la hipotesis i sobre la premisa j

In [ ]:
def predict_matrix(hypotesis_to_be_tested,answers_from_operator):
  M = np.zeros([len(hypotesis_to_be_tested),len(answers_from_operator)])
  
  for i in range(len(hypotesis_to_be_tested)):
      for j in range(len(answers_from_operator)):
        dataset = {'premise': [answers_from_operator[j]], 'hypothesis': [hypotesis_to_be_tested[i]]}

        dataloader = torch.utils.data.DataLoader(TestDataset(pd.DataFrame(dataset)), batch_size=1, shuffle=False, num_workers=1)
        M[i,j] = get_label(dataloader)

  return(M)

*7*. Se crea la función conversation_analysis, que recibe como parámetros una lista de hipótesis a ser testeadas, y las respuestas brindadas por la operadora. Se utiliza como base la función predict_matrix para indicar para cada hipótesis si se encuentra presente en las respuestas, y en el órden correcto. 



In [ ]:
def conversation_analysis(hypotesis_to_be_tested,answers_from_operator):
  out=predict_matrix(hypotesis_to_be_tested,answers_from_operator)
  hypotesis_ordered= np.ones([len(hypotesis_to_be_tested)])*-1
  for h in range(len(hypotesis_to_be_tested)):
    checker= np.zeros([len(hypotesis_to_be_tested)])
    checker[h]=1
    c=out.T.dot(checker)
    for x in range(len(c)):
      if c[x]==0:
        hypotesis_ordered[h]=x
        break

  ok=[]
  wrong=[]

  for i in range(len(hypotesis_ordered)):
    if hypotesis_ordered[i] ==i:
      ok.append(hypotesis_to_be_tested[i] +" correctamente presente en la comunicación")
    elif hypotesis_ordered[i]<0:
      wrong.append(hypotesis_to_be_tested[i] +" NO PRESENTE en la comunicación")
    else:
      wrong.append(hypotesis_to_be_tested[i] +" presente en la comunicación, pero NO EN EL ORDEN CORRECTO")

  print("** Devolución de la comunicación realizado por la operadora **")
  if len(wrong)>0:
    print("A mejorar:")
  else:
    ok.append("Felicitaciones! Comunicación realizada de forma deseada")
  for w in wrong:
    print(w)

  if len(ok)>0:
    print("Positivo:")
  for o in ok:
    print(o)

*8*. **PRUEBA**

*8.1* La lista de hipótesis a testear

In [ ]:
hypotesis_to_be_tested=["Saludo","Ofrecio asistencia"]

*8.2* Se testean 4 casos distintos de respuestas

In [ ]:
answers_from_operator=["Hola", "¿En que puedo ayudar?"]
conversation_analysis(hypotesis_to_be_tested,answers_from_operator)

** Devolución de la comunicación realizado por la operadora **
Positivo:
Saludo correctamente presente en la comunicación
Ofrecio asistencia correctamente presente en la comunicación
Felicitaciones! Comunicación realizada de forma deseada


In [ ]:
answers_from_operator=["Hola", "que cansancio"]
conversation_analysis(hypotesis_to_be_tested,answers_from_operator)

** Devolución de la comunicación realizado por la operadora **
A mejorar:
Ofrecio asistencia NO PRESENTE en la comunicación
Positivo:
Saludo correctamente presente en la comunicación


In [ ]:
answers_from_operator=["¿En que puedo ayudar?","Hola"]
conversation_analysis(hypotesis_to_be_tested,answers_from_operator)

** Devolución de la comunicación realizado por la operadora **
A mejorar:
Saludo presente en la comunicación, pero NO EN EL ORDEN CORRECTO
Ofrecio asistencia presente en la comunicación, pero NO EN EL ORDEN CORRECTO


In [ ]:
answers_from_operator=["muy buenas noches","Cómo se encuentra el día de hoy?"] 
conversation_analysis(hypotesis_to_be_tested,answers_from_operator)

** Devolución de la comunicación realizado por la operadora **
A mejorar:
Ofrecio asistencia NO PRESENTE en la comunicación
Positivo:
Saludo correctamente presente en la comunicación


*8.3* El siguiente código permite una validación un poco más dinámica. Se utiliza la función creada en in 5. Cada vez que el siguiente bloque es ejecutado, se realiza el análisis sobre una nueva salida

In [ ]:
answers_from_operator,answers_from_client=make_conversation() 
conversation_analysis(hypotesis_to_be_tested,answers_from_operator)

Operadora: que tal está?
  Cliente: Hola, mi nombre es Pedro
Operadora: hay muchas consultas hoy
  Cliente: Necesito asistencia con un seguro
** Devolución de la comunicación realizado por la operadora **
A mejorar:
Ofrecio asistencia NO PRESENTE en la comunicación
Positivo:
Saludo correctamente presente en la comunicación
